# Requiment 1

## Library

In [10]:
import pandas as pd

## Data

In [11]:
def read_file(filename):
    with open(filename, 'r') as file:
        text = file.readlines()
    df = pd.DataFrame(text, columns=["Text"])
    return df

## LSH

## Shingling

In [30]:
class InMemoryMinHashLSH:
    def __init__(self, documents):
        self.documents = documents
        self.shingled_data = None
        self.minhashing_data = None
        self.lsh_data = None

    def shingling(self):
      k = 10
      shingling_data = []  # List to store shingled data
      for text in self.documents['Text']:
          shingles = []  # List to store shingles for each document
          for i in range(len(text) - k + 1):
              shingle = text[i:i+k]  # Extract shingle of length k
              shingles.append(shingle)
          shingling_data.append(shingles)
      self.shingled_data = shingling_data  # Store shingled data
      return shingling_data


    def minhashing(self):
        # Your minhashing implementation here
        pass

    def locality_sensity_hashing(self):
        # Your locality sensitive hashing implementation here
        pass

    def run(self):
        # Run the entire process
        print(self.shingling())  # Call shingling method using self
        return

    def approxNearestNeighbors(self, key, n):
        pass


In [31]:
data = read_file("/content/WebOfScience-5736.txt")
minhash_lsh = InMemoryMinHashLSH(data)
minhash_lsh.run()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

